# Segmentace obrazu a měření rozměrů
Základní znalostí při zpracování obrazu je nejenom ideálně nasnímat obraz, ale hlavně ze získaného obrazu dostat použitelné informace. Extrakce objektů z obrazu (oddělení objektů od pozadí) se nazívá segmentace. Nejčastěji se využívá metod prahování, hranové detekce nebo kontur ([viz link](https://circuitdigest.com/tutorial/image-segmentation-using-opencv)).

Když už se podaří získat z obrazu objekty, je dále potřeba změřit jejich rozměry - a to nejen v pixelech, ale hlavně reálné rozměry v cm nebo dokonce v menších jednotkách. 

Na cvičení se bude využívat umělý obrázek s objekty jejichž rozměry bude třeba automatizovaně získat.

![](images/index.jpg)

### Import knihoven a konfigurace

In [ ]:
%run ../svz.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Seznam funkcí pro přehlednost:


První část
- [`show_images(...)`](../svz.ipynb#show_functions)
- [`show_images(...)`](../svz.ipynb#show_functions)
- [`load_image(...)`](../svz.ipynb#load_save_functions)
- [`crop(...)`](../svz.ipynb#preprocessing_functions)

Druhá část
- [`copy_to(...)`](../svz.ipynb#measuring_funtions)
- [`midpoint(...)`](../svz.ipynb#measuring_funtions)
- [`order_points(...)`](../svz.ipynb#measuring_funtions)
- [`rotate_image(...)`](../svz.ipynb#measuring_funtions)
- [`draw_rotated_text(...)`](../svz.ipynb#measuring_funtions)
- [`draw_real_sizes(...)`](../svz.ipynb#measuring_funtions)

---

### Úkol 1

Úkol je zaměřen na segmentace obrazů, získání kontur objektů a měření poměru rozměrů v cm a pixelech.

**1) Načtěte a zkontrolujte zaznamenaný obrázek vzoru pomocí kamery. Nadefinujte proměné s šířkou a výškou referečního objektu.**

Obrázek nastavte tak, aby byl ideálně záběr pouze na něj. *HINT: Je možné pomoci si oříznutím obrázku (crop).*

In [ ]:
img_patterns = ...(...) ###
show_images(img_patterns)

In [ ]:
ref_width_real = ###
ref_height_real = ###

**2) Využijte funkci [cv2.inRange()](https://docs.opencv.org/3.4.1/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981) k získání masky referenčního obdélníku.**

Funkce kromě zdrojového obrázku přijímá dolní a horní mez hodnoty pixelů ve formátu BGR (pozor celé OpenCV funguje v režimu BGR, ne RGB). Tedy dvě trojice (tuple) ve formátu (b, g, r), kde 0 <= b, g, r <= 255. Pixely s hodnotou >= než dolní mez a <= než horní mez budou mít ve výsledné masce hodnotu 255, ostatní 0. 

Dolní a horní mez naleznete experimentálně.

In [ ]:
ref_mask = cv2.inRange(img_patterns, ...) ### tuples
show_images(ref_mask)

**3) Naleznete konturu která pokrývá nevětší plochu, otestujte aproximace kontury na obdélník.**

Výsledná maska by měla ideálně obsahovat pouze referenční obdelník, v obrazu se ale obecně může vyskytovat i šum. 
Nyní je nutné získat pozici referenčního obdelníku v obraze masky. 
Nejjednoduším způsobem je vyhledání kontur (obrysů) v obraze pomocí [cv2.findContours()](https://docs.opencv.org/3.3.1/d3/dc0/group__imgproc__shape.html#ga17ed9f5d79ae97bd4c7cf18403e1689a). Vyhledání kontur funguje jen na binárním černobílém obraze, proto jsme nejdříve museli využít cv2.inRange() k získání masky.

Konturu si lze představit jako křivku spojující několik bodů kolem obrysu souvislého objektu. Funkce cv2.findContours() příjímá navíc dva parametry, contour retrieval mode a contour approximation method. Těmito parametry se ve většině případů nemusíme zabývat. Zjednodušeně je tedy nastavíme `mode=cv2.RETR_LIST` a `method=cv2.CHAIN_APPROX_SIMPLE`. Pokud by vás přeci jen zajímaly, můžete si o nich přečíst [zde](https://docs.opencv.org/3.3.1/d4/d73/tutorial_py_contours_begin.html)

Vzhledem k šumu, který se na každém snímku vyskytuje, prakticky nikdy nenajdeme konturu pouze jednu. Je tedy nutné následně provést filtrování. V našem případě si vystačíme s výběrem kontury, která má největší plochu.

Posledním krokem je validace našeho postupu a vizualizace nalezené kontury pomocí funkce [cv2.drawContours()](https://docs.opencv.org/3.3.1/d6/d6e/group__imgproc__draw.html#ga746c0625f1781f1ffc9056259103edbc).
                                                                                

In [ ]:
contours, _  = cv2.findContours(...) ###
print(f'Found {len(contours)} contours.')

# pick only the contour with the biggest area
contour_biggest = max(contours, key=cv2.contourArea)
print(f'Biggest contour area: {cv2.contourArea(contour_biggest)}, coordinates:\n {contour_biggest.reshape((-1, 2)).tolist()}')
contour_drawn = cv2.drawContours(img_patterns.copy(), [contour_biggest], -1, color=(255, 0, 0), thickness=5)
show_images(contour_drawn)

Z výsledných souřadnic vidíme, že kontura netvoří přesný obdelník, proto ji musíme obdelníkem aproximovat. To lze pomocí funkce [cv2.minAreaRect](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9). Návratovou hodnotou této funkce je tuple - (střed obdélníku (x,y), (výška, šířka), úhel rotace obdélníku). Jedna otázka zní, co je výška a co je šířka? Musíte to vždycky kontrolovat okem.

Pomocná funkce [cv2.boxPoints()](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#gaf78d467e024b4d7936cf9397185d2f5c) převádí nalezený obdelník z formátu (střed obdélníku (x,y), (šířka, výška), úhel rotace obdélníku) na 4 rohové body obdélníku ve formátu (x, y). To se může hodit např. k vizualizaci (lze použít jako vstup pro cv2.drawContours()). Pozor, pořadí vrácených bodů není zaručeno.

In [ ]:
rect = cv2.minAreaRect(contour_biggest) ###
print(f'Rect tuple: {rect}')
print(f'(cx, cy)={rect[0]}, (width, height)={rect[1]}, angle={rect[2]}')
print(f'Rect points: {cv2.boxPoints(rect).tolist()}')

**5) Nalezenou šířku referenčního obdelníku v pixelech můžeme konečně využít k získání poměru skutečné šířky obdélníku v reálných jednotkách a pixelové šířky obdélníku v obraze. Tento poměr budeme následně potřebovat k výpočtu rozměrů ostatních neznámých objektů.**

In [ ]:
ref_width_image, ref_height_image = ... ###
real_image_ratio = ... ### 
print(f'Ratio between real width and image width: {real_image_ratio}')

---

### Úkol 2

Úkol 1 máme za sebou a už víme poměr mezi rozměrem v cm a pixelech, tím pádem už můžeme naměřit rozměry ostatních objektů na obrázku s neznámými rozměry.

**1) Opět využijte funkci [cv2.inRange()](https://docs.opencv.org/3.4.1/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981), tentokrát k získání masky objektů s neznámými skutečnými rozměry.**

**2) Nalezněte v nové masce snímku kontury stejně jako v případě referenčního objektu.**

**3) Následně je provedeno filtrování kontur podle jejich obsahu. Prahovou hodnotu obsahu v pixelech (threshold) je nutno zvolit experimentálně.**

In [ ]:
patterns_mask = ... ###
show_images(patterns_mask)

In [ ]:
contours, _  = ...
print(f'Found {len(contours)} contours.')

# Filter out noise
threshold = ... ###
contours =  [c for c in contours if cv2.contourArea(c) > threshold]
print(f'After filtering, {len(contours)} contours remained.')

contour_drawn = cv2.drawContours(img_patterns.copy(), contours, -1, color=(255, 0, 0 ), thickness=10)
show_images(contour_drawn)

# Sort contours by area. Just for better debugging.
contours.sort(key=cv2.contourArea, reverse=True)

**4) Nalezněte skutečné rozměry objektu**

Nyní potřebujeme pro jednotlivé kontury zjistit jejich skutečné rozměry, které chceme vizualizovat do výsledného obrázku.

Pro každou konturu tedy získáme jeji obdélníkovou aproximaci pomocí [cv2.minAreaRect](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9). Následně můžeme vypočítat skutečnou šířku a výšku objektu, díky předešlému vypočítanému poměru mezi skutečnou šířkou a pixelovou šířkou. Posledním krokem je volání funkce `draw_real_sizes()`, která se do vstupního obrázku pokusí vykreslit rozměry nalezeného objektu.

In [ ]:
# create a copy of original image
sizes_drawn = img_patterns.copy()

for c in contours:
    rect = ... ###
    shape_width, shape_height = ... ### 
    real_width = ...  ###
    real_height = ... ###
    
    cv2.drawContours(sizes_drawn, [c], -1, color=(255, 0, 0 ), thickness=5)
    sizes_drawn = draw_real_sizes(sizes_drawn, rect, real_height, real_width,
                                  lbl_size_scale=.7, lbl_color=(0, 0, 255), lbl_thickness=1) ###

show_images(sizes_drawn)